In [7]:
import descarteslabs as dl
from datetime import datetime
# from shapely.geometry import Point
# import osgeo
# from osgeo import ogr, osr
from subprocess import check_output
import subprocess 
from sys import argv

## Open File

In [17]:
process = subprocess.call('mkdir temp', shell=True)

In [6]:
args = ['gsutil cp gs://dl-fires/inciweb-data/conus_fires_20180517-18UTC.txt temp/']
p = subprocess.Popen(args, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print p.communicate()

('', 'Copying gs://dl-fires/inciweb-data/conus_fires_20180517-18UTC.txt...\n/ [0 files][    0.0 B/  1.0 KiB]                                                \r/ [1 files][  1.0 KiB/  1.0 KiB]                                                \r\nOperation completed over 1 objects/1.0 KiB.                                      \n')


In [24]:
def iterate_over_file(string):
    counter = 0
    for line in iter(string.splitlines()):
        if counter == 0:
            counter +=1
        else: 
            initiate_fire_instance(line)
            counter +=1

In [46]:
# This could be refactored to include a class 
def initiate_fire_instance(fire_string): 
    fire_arr = fire_string.split()
    name_list = fire_arr[:-4]
    # print(' '.join([str(x) for x in name_list]))
    name = ' '.join([str(x) for x in name_list])
    fire_id = fire_arr[-4]
    lat = fire_arr[-3]
    lon = fire_arr[-2]
    coords=[lat,lon]
    acres = fire_arr[-1]
#     print("The {} has burned for {} acres, is at {} and has the id {}".format(name,acres,coords,fire_id))

In [47]:
txt = open('temp/conus_fires_20180517-18UTC.txt')
file_string_contents = txt.read()
iterate_over_file(file_string_contents)

The Mallard Fire has burned for nan acres, is at ['34.81', '-101.31'] and has the id id:5790
The Pinery Fire has burned for 1200.00 acres, is at ['31.98', '-109.37'] and has the id id:5795
The AVIAN COMPLEX has burned for nan acres, is at ['25.90', '-81.31'] and has the id id:5750
The 2018 Shasta-Trinity Lightning Fires has burned for nan acres, is at ['40.90', '-123.16'] and has the id id:5789
The Woods Fire has burned for nan acres, is at ['34.34', '-110.97'] and has the id id:5794
The Viewpoint Fire has burned for 5600.00 acres, is at ['34.69', '-112.37'] and has the id id:5791
The McDannald Fire has burned for nan acres, is at ['30.55', '-104.24'] and has the id id:5773
The Tinder Fire has burned for nan acres, is at ['34.56', '-111.16'] and has the id id:5763
The Grape Fire has burned for 244.00 acres, is at ['40.71', '-123.55'] and has the id id:5761
The Gate Mountain Fire has burned for 1040.00 acres, is at ['38.62', '-79.01'] and has the id id:5744
The Tye River Fire has burned

## Read Fire Directories 
Create a new one for the output if it does not exist 

In [70]:
args = ['gsutil ls gs://dl-fires/imagery']
p = subprocess.Popen(args, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
buckets= p.communicate()[0]
buckets = buckets.split('\ngs://dl-fires/imagery/')
buckets = [bucket.replace('/', '') for bucket in buckets]
buckets = [bucket.replace('\n', '') for bucket in buckets]
buckets = buckets[1:]
buckets

['test_fire', 'test_fire_2']

## Find imagery

In [9]:
def make_tile(coords): 
    # to get the tile centered around your lat/lon, give it a tilesize of 1 and a pad of the half-width of your desired image (in meters)
    print coords[0]
    print coords[1]
    bounding_box = dl.raster.dltile_from_latlon(
        lat=coords[0],
        lon=coords[1], 
        resolution=30, 
        tilesize=2, 
        pad=600)

    return bounding_box
    

In [3]:
def search_products(date, geometry):
    print(date)
    from pprint import pprint 
    pprint(geometry['geometry']['coordinates'])
    images = dl.metadata.search(
        ['landsat:LC08:01:RT:TOAR', 'sentinel-2:L1C'],
        geom= geometry, 
        start_datetime=date, 
        end_datetime= datetime.today().isoformat()
        )
    pprint("The number of total images found is {}.".format(len(images['features'])))
    return [image['id'] for image in images['features']]

In [7]:
def download_imagery(ids, geometry):
    for image_id in ids: 
        print image_id
        dl.raster.raster(
                        image_id,
                        bands=['swir2', 'swir1', 'nir', 'alpha'],
                        scales=[[0,4000], [0, 4000], [0, 4000], None],
                        data_type='Byte',
                        cutline=geometry,
                        save= True,
                        outfile_basename=image_id)

In [5]:
def find_imagery(fire_name, date, coords, footprint=None):
    # Where name is a str, coords is an array of [lat,long], and footprint is an optional geometry
#     if footprint:
#         image_ids = search_products(date,footprint)
#         download_imagery(image_ids, footprint)
#     else:
    dl_tile = make_tile(coords)
    image_ids = search_products(date, dl_tile)
    download_imagery(image_ids, dl_tile)

Driver Code

In [8]:
find_imagery('Test Fire', '2018-05-10',[36.87, -120.24])         
          

36.87
-120.24
2018-05-10
[
  [
    [-120.32835512645352, 36.80264609569112],
    [-120.15637285166773, 36.798655452355234],
    [-120.1512387673139, 36.936948905000946],
    [-120.32353061867214, 36.940959542556456],
    ...
  ]
]
'The number of total images found is 2.'
sentinel-2:L1C:2018-05-12_10SGG_99_S2A_v1
sentinel-2:L1C:2018-05-12_11SKA_99_S2A_v1
